In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load datasets
train_features = pd.read_csv("DengAI_train.csv")
train_labels = pd.read_csv("DengAI_Predicting_Disease_Spread_-_Training_Data_Labels.csv")
test_features = pd.read_csv("DengAI_test.csv")

# Merge train features and labels
train_df = pd.merge(train_features, train_labels, on=["city", "year", "weekofyear"])

# Forward fill missing values (time-series)
train_df.fillna(method='ffill', inplace=True)
test_features.fillna(method='ffill', inplace=True)

# Drop 'week_start_date' since it's non-numeric
train_df.drop(columns=['week_start_date'], inplace=True)
test_features.drop(columns=['week_start_date'], inplace=True)

# Normalize numeric features
scaler = StandardScaler()
X_all = pd.concat([train_df.drop(columns=['total_cases']), test_features], axis=0)
X_scaled = scaler.fit_transform(X_all.select_dtypes(include=[np.number]))

# Restore DataFrame with scaled values
X_all_scaled = pd.DataFrame(X_scaled, columns=X_all.select_dtypes(include=[np.number]).columns)
X_all_scaled[['city', 'year', 'weekofyear']] = X_all[['city', 'year', 'weekofyear']].reset_index(drop=True)

# Re-split
X_train = X_all_scaled.iloc[:len(train_df)]
X_test = X_all_scaled.iloc[len(train_df):]
y_train = train_df['total_cases']


<ipython-input-29-7fa4144b01b5>:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_df.fillna(method='ffill', inplace=True)
<ipython-input-29-7fa4144b01b5>:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_features.fillna(method='ffill', inplace=True)


In [31]:
# 🧠 优化版 GRU 模型：分别为 SJ 和 IQ 建模，目标降低 RMSE 和 MAE

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def run_gru_model(city_code, window):
    print(f"\n🏙️ Running GRU for city: {city_code.upper()} (window={window})")

    # Step 1: Load and merge data
    features_df = pd.read_csv('DengAI_train.csv')
    labels_df = pd.read_csv('DengAI_Predicting_Disease_Spread_-_Training_Data_Labels.csv')
    df = pd.merge(features_df, labels_df, on=['city', 'year', 'weekofyear'])

    # Step 2: Filter city and clean
    df = df[df['city'] == city_code].copy()
    df.reset_index(drop=True, inplace=True)
    df.drop(columns=['week_start_date', 'city'], inplace=True)
    df = df.ffill()

    # Step 3: Scale features
    scaler = StandardScaler()
    X = df.drop(columns=['total_cases'])
    X_scaled = scaler.fit_transform(X)
    y = np.log1p(df['total_cases'])  # log transform

    # Step 4: Create sequences
    def create_sequences(X, y, window):
        Xs, ys = [], []
        for i in range(window, len(X)):
            Xs.append(X[i-window:i])
            ys.append(y[i])
        return np.array(Xs), np.array(ys)

    X_seq, y_seq = create_sequences(X_scaled, y, window=window)
    split = int(0.8 * len(X_seq))
    X_train, X_test = X_seq[:split], X_seq[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    if len(X_train) == 0 or len(X_test) == 0:
        print("❌ Not enough data to train GRU for this city.")
        return

    # Step 5: Build deep GRU model
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = GRU(128, return_sequences=True)(input_layer)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)
    x = GRU(64)(x)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)
    output = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0002), loss='mse')

    # Step 6: Train with early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=200, batch_size=16, validation_split=0.1,
              callbacks=[early_stop], verbose=0)

    # Step 7: Predict & evaluate
    y_pred_log = model.predict(X_test).flatten()
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_test)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1))) * 100

    print(f"📊 {city_code.upper()} GRU Model Results:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")

# Run both cities with optimal window sizes
run_gru_model('sj', window=12)  # San Juan
run_gru_model('iq', window=8)   # Iquitos



🏙️ Running GRU for city: SJ (window=12)
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
📊 SJ GRU Model Results:
MAE: 19.46
RMSE: 34.80
MAPE: 107.70%

🏙️ Running GRU for city: IQ (window=8)
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
📊 IQ GRU Model Results:
MAE: 7.48
RMSE: 12.92
MAPE: 121.22%


In [32]:
# 🧠 LSTM深度学习模型：分别建模 San Juan (sj) 和 Iquitos (iq)

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def run_lstm_model(city_code, window):
    print(f"\n🏙️ Running LSTM for city: {city_code.upper()} (window={window})")

    # Step 1: Load and merge data
    features_df = pd.read_csv('DengAI_train.csv')
    labels_df = pd.read_csv('DengAI_Predicting_Disease_Spread_-_Training_Data_Labels.csv')
    df = pd.merge(features_df, labels_df, on=['city', 'year', 'weekofyear'])

    # Step 2: Filter city and clean
    df = df[df['city'] == city_code].copy()
    df.reset_index(drop=True, inplace=True)  # ← 关键修复
    df.drop(columns=['week_start_date', 'city'], inplace=True)
    df = df.ffill()

    # Step 3: Scale features
    scaler = StandardScaler()
    X = df.drop(columns=['total_cases'])
    X_scaled = scaler.fit_transform(X)
    y = np.log1p(df['total_cases'])

    # Step 4: Create sequences
    def create_sequences(X, y, window):
        Xs, ys = [], []
        for i in range(window, len(X)):
            Xs.append(X[i-window:i])
            ys.append(y[i])
        return np.array(Xs), np.array(ys)

    X_seq, y_seq = create_sequences(X_scaled, y, window=window)
    split = int(0.8 * len(X_seq))
    X_train, X_test = X_seq[:split], X_seq[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    if len(X_train) == 0 or len(X_test) == 0:
        print("❌ Not enough data to train LSTM for this city.")
        return

    # Step 5: Build LSTM model
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = LSTM(128, return_sequences=True)(input_layer)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)
    x = LSTM(128)(x)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)
    output = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0003), loss='mse')

    # Step 6: Train
    early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=150, batch_size=16, validation_split=0.1,
              callbacks=[early_stop], verbose=0)

    # Step 7: Evaluate
    y_pred_log = model.predict(X_test).flatten()
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_test)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1))) * 100

    print(f"📊 {city_code.upper()} Model Results:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")

# 分别运行 San Juan（12周窗口）和 Iquitos（6周窗口）
run_lstm_model('sj', window=12)
run_lstm_model('iq', window=6)


🏙️ Running LSTM for city: SJ (window=12)
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
📊 SJ Model Results:
MAE: 17.37
RMSE: 31.29
MAPE: 88.72%

🏙️ Running LSTM for city: IQ (window=6)
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
📊 IQ Model Results:
MAE: 7.85
RMSE: 11.98
MAPE: 236.03%
